In [ ]:
from pathlib import Path

import geopandas as gpd
import pyarrow.parquet as pq
from irv_datapkg import crop_raster
from tqdm.auto import tqdm
tqdm.pandas()


In [ ]:
iso = "BWA"
save_path = Path("/home/mert2014/projects/bhutan-demo") / iso
save_path.mkdir(exist_ok=True, parents=True)
path_prefix = str(save_path).replace("~", "/home/mert2014")
cluster_path = Path("/mnt/linux-filestore/mert2014/projects/open-gira")
hazard_dirs = Path("/mnt/linux-filestore/mert2014/projects/open-gira/results/input/").glob("hazard-*")
lcnr_path = Path("/mnt/linux-filestore/LCNR_DR/GLOBAL/300m_9s/RestorationCostBenefit")

## Extract admin bounds, hydrobasins

In [ ]:
adm0 = gpd.read_parquet("../results/input/admin-boundaries/gadm36_levels.parquet/level=0/data.parquet", filters=[('GID_0', '==', iso)])
adm0.to_file(save_path / f"adm0_{iso}.gpkg", engine="pyogrio")

In [ ]:
bb0 = gpd.read_file(save_path / f"adm0_{iso}.gpkg", engine="pyogrio")
bbgeom = bb0.geometry.iloc[0]
bb0.bounds

In [ ]:
hybas = gpd.read_parquet("../results/input/hydrobasins/hybas_lev12_v1c.geoparquet")

In [ ]:
bbox = bb0.bounds.iloc[0]
hybas = hybas.cx[bbox.minx:bbox.maxx, bbox.miny:bbox.maxy].copy()

In [ ]:
hybas.to_file(save_path / "hybas_lev12_v1c.gpkg", engine="pyogrio")

In [ ]:
adm0 = gpd.read_parquet("../results/input/admin-boundaries/gadm36_levels.parquet/level=0/data.parquet")
adm1 = gpd.read_parquet("../results/input/admin-boundaries/gadm36_levels.parquet/level=1/data.parquet")
adm2 = gpd.read_parquet("../results/input/admin-boundaries/gadm36_levels.parquet/level=2/data.parquet")
adm0 = adm0.cx[bbox.minx:bbox.maxx, bbox.miny:bbox.maxy]
adm1 = adm1.cx[bbox.minx:bbox.maxx, bbox.miny:bbox.maxy]
adm2 = adm2.cx[bbox.minx:bbox.maxx, bbox.miny:bbox.maxy]
adm0.to_file(save_path / "adm0.gpkg", engine="pyogrio")
adm1.to_file(save_path / "adm1.gpkg", engine="pyogrio")
adm2.to_file(save_path / "adm2.gpkg", engine="pyogrio")

In [ ]:
# Extract opportunity areas for landslide, mangrove; planting costs and biodiversity, carbon benefits

# Landslide, general cost and benefit
for slug in ["G_BioBenefit_9s", "G_CarbonBenefit_9s", "G_PlantingCost_9s", "G_RegenCost_9s", "G_LandslideNbS_123_9s", "G_PotentialNonCoastalTreeNBS_9s"]:
        in_fname = lcnr_path / "Final Global Outputs" / f"{slug}.tif"
        out_fname = save_path / f"{slug}.tif"
        crop_raster(in_fname, out_fname, bbgeom)

# Mangrove
in_fname = lcnr_path / "Final Mangrove Outputs/ManRestorClass_9s.tif"
out_fname = save_path / f"ManRestorClass_9s.tif"
crop_raster(in_fname, out_fname, bbgeom)

In [ ]:
for d in tqdm(hazard_dirs):
    print(d.name)
    out_dir = save_path / d.name
    out_dir.mkdir(parents=True, exist_ok=True)
    for in_fname in (d / "planet-latest").glob("*.tif"):
        print(d.name, in_fname)
        out_fname = out_dir / in_fname.name
        crop_raster(in_fname, out_fname, bbgeom)

# Set up extracted baseline EAD splits

In [ ]:
def extract_pq(pds_dir, read_columns, iso, out_gpkg):
    pds = pq.ParquetDataset(pds_dir)
    nonempty = []
    schemas = []
    for pf_name in pds.files:
        try:
            pf = pq.ParquetFile(pf_name)
            if "id" in pf.schema.names:
                nonempty.append(pf_name)
        except:
            continue
        for s in schemas:
            if s.equals(pf.schema):
                break
        else:
            schemas.append(pf.schema)
    # print(schemas)
    filters = []
    if iso is not None:
        filters.append(('from_iso_a3', '==', iso))
    else:
        filters=None
    pt = pq.read_table(
        nonempty,
        columns=read_columns,
        filters=filters
    )
    df = pt.to_pandas()
    df.geometry = gpd.GeoSeries.from_wkb(df.geometry)
    gdf = gpd.GeoDataFrame(df).set_crs(epsg=4326)
    gdf.to_file(out_gpkg, driver="GPKG", engine="pyogrio")

In [ ]:
ls_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-road-tertiary/hazard-landslide-arup/EAD_and_cost_per_trigger'
ls_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    "hazard-_landslide_sum__road_lower_EAD",
    "hazard-_landslide_sum__road_damage_fraction_EAD",
    "hazard-_landslide_sum__road_upper_EAD",
]
ls_out_gpkg = save_path / "landslide_road_damages.gpkg"
extract_pq(ls_pds_dir, ls_read_columns, iso, ls_out_gpkg)

In [ ]:
ls_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-landslide-arup/EAD_and_cost_per_trigger'
ls_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    "hazard-_landslide_sum__rail_lower_EAD",
    "hazard-_landslide_sum__rail_damage_fraction_EAD",
    "hazard-_landslide_sum__rail_upper_EAD",
]
ls_out_gpkg = save_path / "landslide_rail_damages.gpkg"
extract_pq(ls_pds_dir, ls_read_columns, iso, ls_out_gpkg)

In [ ]:
ls_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-landslide-arup/split_EAD_and_cost_per_trigger'
ls_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    "hazard-_landslide_sum__rail_lower_EAD",
    "hazard-_landslide_sum__rail_damage_fraction_EAD",
    "hazard-_landslide_sum__rail_upper_EAD",
]
ls_out_gpkg = save_path / "landslide_rail_damages_split.gpkg"
extract_pq(ls_pds_dir, ls_read_columns, iso, ls_out_gpkg)

In [ ]:
rf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-road-tertiary/hazard-aqueduct-river/EAD'
rf_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    'hazard-inunriver_historical_MAX_1980_EAD',
    'hazard-inunriver_historical_MEAN_1980_EAD',
    'hazard-inunriver_historical_MIN_1980_EAD',
    'hazard-inunriver_rcp4p5_MAX_2050_EAD',
    'hazard-inunriver_rcp4p5_MEAN_2050_EAD',
    'hazard-inunriver_rcp4p5_MIN_2050_EAD',
    'hazard-inunriver_rcp8p5_MAX_2050_EAD',
    'hazard-inunriver_rcp8p5_MEAN_2050_EAD',
    'hazard-inunriver_rcp8p5_MIN_2050_EAD',
]
rf_out_gpkg = save_path / "river-aqueduct_road_damages.gpkg"
extract_pq(rf_pds_dir, rf_read_columns, iso, rf_out_gpkg)

rf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-aqueduct-river/EAD'
rf_out_gpkg = save_path / "river-aqueduct_rail_damages.gpkg"
extract_pq(rf_pds_dir, rf_read_columns, iso, rf_out_gpkg)

In [ ]:
cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-road-tertiary/hazard-aqueduct-coast/EAD'
cf_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    'hazard-inuncoast_historical_MAX_hist_0_EAD',
    'hazard-inuncoast_historical_MEAN_hist_0_EAD',
    'hazard-inuncoast_historical_MIN_hist_0_EAD',
    'hazard-inuncoast_rcp4p5_MAX_2050_0_EAD',
    'hazard-inuncoast_rcp4p5_MEAN_2050_0_EAD',
    'hazard-inuncoast_rcp4p5_MIN_2050_0_EAD',
    'hazard-inuncoast_rcp8p5_MAX_2050_0_EAD',
    'hazard-inuncoast_rcp8p5_MEAN_2050_0_EAD',
    'hazard-inuncoast_rcp8p5_MIN_2050_0_EAD',
]
cf_out_gpkg = save_path / "coastal-aqueduct_road_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)

cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-aqueduct-coast/EAD'
cf_out_gpkg = save_path / "coastal-aqueduct_rail_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)


In [ ]:
cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-road-tertiary/hazard-coastal-deltares/EAD'
cf_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    "hazard-GFM_MERITDEM1km_2018_EAD",
    "hazard-GFM_NASADEM1km_2018_EAD",
    "hazard-GFM_MERITDEM1km_2050_EAD",
    "hazard-GFM_NASADEM1km_2050_EAD",
]
cf_out_gpkg = save_path / "coastal-deltares_road_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)

cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-coastal-deltares/EAD'
cf_out_gpkg = save_path / "coastal-deltares_rail_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)

In [ ]:
cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-road-tertiary/hazard-river-jrc/EAD'
cf_read_columns = [
    "id",
    "geometry",
    "from_iso_a3",
    "to_iso_a3",
    "asset_type",
    "osm_way_id",
    "bridge",
    "hazard-floodMapGL_EAD"
]
cf_out_gpkg = save_path / "river-jrc_road_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)

cf_pds_dir = cluster_path / 'results/direct_damages/planet-latest_filter-rail/hazard-river-jrc/EAD'
cf_out_gpkg = save_path / "river-jrc_rail_damages.gpkg"
extract_pq(cf_pds_dir, cf_read_columns, iso, cf_out_gpkg)

# Extract opportunities

In [ ]:
def extract_opp(pds_dir, iso, out_gpkg):
    pds = pq.ParquetDataset(pds_dir)
    nonempty = []
    schemas = []
    for pf_name in pds.files:
        pf = pq.ParquetFile(pf_name)
        if "HYBAS_ID" in pf.schema.names:
            nonempty.append(pf_name)

        for s in schemas:
            if s.equals(pf.schema):
                break
        else:
            schemas.append(pf.schema)
    # print(schemas)
    filters = []
    if iso is not None:
        filters.append(('GID_0', '==', iso))
    else:
        filters=None
    pt = pq.read_table(
        nonempty,
        filters=filters
    )
    df = pt.to_pandas()
    df.geometry = gpd.GeoSeries.from_wkb(df.geometry)
    gdf = gpd.GeoDataFrame(df).set_crs(epsg=4326)
    gdf.to_file(out_gpkg, driver="GPKG", engine="pyogrio")

In [ ]:
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/landslide_slope_vegetation.parquet")),
    iso,
    save_path / "landslide_slope_vegetation.gpkg",
)
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/mangrove.parquet")),
    iso,
    save_path / "mangrove.gpkg",
)
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/river_basin_afforestation.parquet")),
    iso,
    save_path / "river_basin_afforestation.gpkg",
)
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/landslide_slope_vegetation_with_EAD.parquet")),
    iso,
    save_path / "landslide_slope_vegetation_with_EAD.gpkg",
)
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/mangrove_with_EAD.parquet")),
    iso,
    save_path / "mangrove_with_EAD.gpkg",
)
extract_opp(
    list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/river_basin_afforestation_with_EAD.parquet")),
    iso,
    save_path / "river_basin_afforestation_with_EAD.gpkg",
)

In [ ]:
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/landslide_slope_vegetation.parquet")),
#     None,
#     save_path.parent / "landslide_slope_vegetation.gpkg",
# )
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/mangrove.parquet")),
#     None,
#     save_path.parent / "mangrove.gpkg",
# )
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/river_basin_afforestation.parquet")),
#     None,
#     save_path.parent / "river_basin_afforestation.gpkg",
# )
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/landslide_slope_vegetation_with_EAD.parquet")),
#     None,
#     save_path.parent / "landslide_slope_vegetation_with_EAD.gpkg",
# )
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/mangrove_with_EAD.parquet")),
#     None,
#     save_path.parent / "mangrove_with_EAD.gpkg",
# )
# extract_opp(
#     list((cluster_path / 'results/slices/planet-latest_nbs').glob("slice-*/river_basin_afforestation_with_EAD.parquet")),
#     None,
#     save_path.parent / "river_basin_afforestation_with_EAD.gpkg",
# )